## Task Generator

In this notebook, we will make changes to support a task generator for user simulation study in Tool Sanbbox. The user simulation study has been broken down into two components:
- Natural Language Task Generator: We will take the current randomized state of the environment, all the available tools and prepare a prompt for the model to generate a natural language task.
- User Trajectory Generator: This class will take in a natural language task and try to complete the task in the tool sandbox. Most importantly, we will strip the "thinking" outputs from the outputs of this class. So ideal trajectory should look like a sequence of tool calls and system messages to user.


Additionally, we want to evaluate the *feasibility* of the task generator and *accuracy* of the trajectory generator. Feasbility can be calculated as *how many models are able to complete the task* and accuracy of trajectory generator, we should be able to judge by using milestones and minefields. Milestones and minefields should also be automatically generated but then we also have to find a way to judge it's own accuracy. I know LLM as a judge should be doable but still.

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from dotenv import load_dotenv
import random
from tool_sandbox.common.execution_context import ExecutionContext
from tool_sandbox.scenarios.openai_task_generator import OpenAITaskGenerator
from tool_sandbox.common.tool_discovery import ToolBackend

load_dotenv()

execution_context = ExecutionContext()


In [ ]:
# Create OpenAI task generator
task_generator = OpenAITaskGenerator(
    model_name="o3-mini",
    execution_context=execution_context,
    max_retries=3,
    populate_sample_data=False  # Skipping for MVP
)


In [ ]:
generated_tasks = []
tool_categories = ["contact", "messaging", "reminder", "setting", "utilities"]

for i in range(10):
    num_categories = random.randint(3, len(tool_categories))
    allowed_categories = random.sample(tool_categories, num_categories)
    generated_task = task_generator.generate_task(
        allowed_tool_categories=allowed_categories,
        preferred_tool_backend=ToolBackend.DEFAULT,
    )
    generated_tasks.append(generated_task)


In [ ]:
# dump the generated tasks to a jsonl file
import json
from attrs import asdict
filename = "../data/o3_mini_generated_tasks_new.jsonl"
with open(filename, 'w', encoding='utf-8') as f:
    for task in generated_tasks:
        # Convert the @define object to a dictionary
        task_dict = asdict(task)
        # Write each task as a JSON line
        f.write(json.dumps(task_dict) + '\n')


In [ ]:
import csv
from pathlib import Path
def jsonl_to_csv(jsonl_file: str, csv_file: str) -> None:
    """Convert a JSONL file of generated tasks to CSV format.

    Args:
        jsonl_file: Path to the input JSONL file
        csv_file: Path to the output CSV file
    """
    # Load tasks from JSONL
    tasks = []
    with open(jsonl_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                task_dict = json.loads(line.strip())
                tasks.append(task_dict)

    if not tasks:
        print(f"No tasks found in {jsonl_file}")
        return

    # Write to CSV (excluding task_id)
    fieldnames = ['description', 'required_tools', 'tools_category']

    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

        for task in tasks:
            # Convert lists to semicolon-separated strings for CSV compatibility
            csv_row = {
                'description': task['description'],
                'required_tools': '; '.join(task['required_tools']),
                'tools_category': '; '.join(task['tools_category'])
            }
            writer.writerow(csv_row)

    print(f"✅ Converted {len(tasks)} tasks from {jsonl_file} to {csv_file}")


In [ ]:
jsonl_to_csv("../data/o3_mini_generated_tasks_new.jsonl", "../data/o3_mini_generated_tasks_new.csv")


# Trajectory Parser

- Parses tool trace trajectory from the completed tool sandbox trajectories.
- Will be used for inferencing the models for goal prediction

In [ ]:
from pprint import pprint
from tool_sandbox.analysis.trajectory_parser import parse_trajectory_for_goal_inference

trajectory_path = "../data/agent_o3-mini_user_gpt-4o-2024-05-13_08_14_2025_14_30_58/trajectories/dinner_plan_review_and_confirmation/"

parsed_trajectory = parse_trajectory_for_goal_inference(trajectory_path)


In [ ]:
pprint(parsed_trajectory.steps[2])


# Goal Inference Model calls

In [ ]:
from tool_sandbox.models.goal_inference_agent import GoalInferenceAgent

agent = GoalInferenceAgent(model_name="o3-mini")

results = agent.run_goal_inference_on_trajectory_path("../data/agent_o3-mini_user_gpt-4o-2024-05-13_08_14_2025_14_30_58/trajectories/dinner_plan_review_and_confirmation/")



In [ ]:
for step in results.steps:
    print(f"Message {step.tool_call.message_index}: {step.tool_call.python_function_string}")
    if step.prediction:
        print(f"Prediction: {step.prediction.prediction_type}, {step.prediction.content}")
